## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
#from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all

from functions.aux import MetricTotalAccuraccy
# Load tree transformation
from functions.transform_inference import TransformerInference

from net import NeoNet

class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


IS CUDA AVAILABLE: True


131072

### Basic parameters

In [2]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.2_subqueries'
model_name = 'Model_Adam'
model_path = "./" + model_name + '.pt'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"
optimizer = "Adam"
#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
#min_time_or = 15
#max_time_or = 80
#percent_of_data = 1
#min_time = 5
#max_time = 80


In [3]:
epsilon = 0.75 ## EPSILON
lr_rl = 0.9 ## LEARNING RATE RL
disc_fac = 0.5 # DISCOUNT FACTOR RL 
epoch_rl = 200 ### TOTAL EPOCH POR DATA RL
n = "_" + model_name + "_gamma_" + str(disc_fac) + "_"##SALT Nombre archivos

### Features

In [4]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

In [5]:
ds_train = pd.read_csv(URL + csv_name + '_ds_train_ft.csv', engine='python', encoding='utf-8')
ds_val = pd.read_csv(URL + csv_name + '_ds_val_ft.csv', engine='python', encoding='utf-8')
ds_test = pd.read_csv(URL + csv_name + '_ds_test_ft.csv', engine='python', encoding='utf-8')
ds_rl = pd.read_csv(URL + csv_name + '_ds_rl_ft.csv', engine='python', encoding='utf-8')

In [6]:
print("ds_train.shape",ds_train.shape)
print("ds_val.shape",ds_val.shape)
print("ds_test.shape",ds_test.shape)
print("ds_rl.shape",ds_rl.shape)

ds_train.shape (7938, 15)
ds_val.shape (1641, 15)
ds_test.shape (1072, 15)
ds_rl.shape (2000, 67)


# FUNCTIONS

In [7]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [8]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

Size Train: 7938, Val 1641


In [9]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [10]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (7938, 15)
shape ds_val  : (1641, 15)
shape ds_test : (1072, 15)
shape ds_rl : (2000, 67)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (1641, 5)
shape x_train_query: (7938, 5)
----------x_plan_level_data----------
shape x_val_tree  : (1641,)
shape x_train_tree: (7938,)
----------y_data------------
shape y_val  : (1641,)
shape y_train: (7938,)

----------TEST DATA----------
shape x_test_tree : (1072,)
shape x_test_query: (1072, 5)
shape y_test      : (1072,)
-----------------------

----------RL DATA----------
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


In [11]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

0

In [12]:
data_inference = {}
bao_base = ""
with (open(bao_base + "preprocess_inference_info_" + model_name + ".pickle", "rb")) as openfile:
    while True:
        try:
            data_inference = pickle.load(openfile)
        except EOFError:
            break
# Load Scaler for query features.
scalerx = None
with (open(bao_base + "scalerx_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            scalerx = pickle.load(openfile)
        except EOFError:
            break

pipeline_inverse = None
with (open(bao_base + "pipeline_inverse_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            pipeline_inverse = pickle.load(openfile)
        except EOFError:
            break
with open('NeoNetValues_' + model_name +'.txt', 'r') as file:
    data = file.read().rstrip()
neo_data = ast.literal_eval(data)

IS CUDA AVAILABLE: True


In [13]:
def ActivationFuncReluOrLeaky(string):
    if string == "<class 'torch.nn.modules.activation.LeakyReLU'>":
        return nn.LeakyReLU
    elif string == "<class 'torch.nn.modules.activation.ReLU'>":
        return nn.ReLU
io_dim_model_data = int(neo_data['io_dim'])
query_input_size = int(neo_data['query_input_size'])
query_hidden_inputs = ast.literal_eval(neo_data['query_hidden_inputs'])
query_output = int(neo_data['query_output'])
tree_units = ast.literal_eval(neo_data['tree_units'])
tree_units_dense = ast.literal_eval(neo_data['tree_units_dense'])
tree_activation_tree = ActivationFuncReluOrLeaky(neo_data['activation_tree'])
tree_activation_dense = ActivationFuncReluOrLeaky(neo_data['activation_dense'])

def get_prediction_dataset_from_load(ds, list_columns):
    x_tree = ds["trees"].values
    x_query = ds[list_columns]
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    targets = list(ds["time"].values)
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}

def get_prediction_from_load(x_tree, x_query, targets):
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}


In [14]:
reg = NeoNet(
    io_dim_model_data,
    query_input_size,
    query_hidden_inputs,
    query_output=query_output,
    tree_units=tree_units,
    tree_units_dense=tree_units_dense,
    activation_tree=tree_activation_tree,
    activation_dense=tree_activation_dense,
)
device = torch.device("cuda")
# Loading model parameters.
reg_path = model_path
#reg.load_state_dict(torch.load(reg_path, map_location=torch.device('cpu')))
reg.load_state_dict(torch.load(reg_path))
reg.to(device)
transformer = TransformerInference(
        model=reg,
        tree_transform=data_inference["tree_transform"],
        pipeline_inverse=pipeline_inverse,
        scalerx=scalerx,
        maxcardinality=data_inference["maxcardinality"]
    )
print("Modelos Listos")

Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(1353, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias=True)]
Modelos Listos


In [15]:
 ds_test['time']

0         1.687920
1         1.678266
2         1.109980
3         1.858232
4         1.598426
           ...    
1067    107.000000
1068    101.000000
1069    123.000000
1070    120.000000
1071    138.003726
Name: time, Length: 1072, dtype: float64

In [16]:
results = get_prediction_from_load(ds_test['trees'].values, ds_test[list_columns], ds_test['time'])

In [17]:
results

{'pred': [12.682406425476074,
  4.89743709564209,
  14.758463859558105,
  14.758463859558105,
  17.954404830932617,
  1.592860221862793,
  14.758463859558105,
  1.5922574996948242,
  37.72126007080078,
  14.75821590423584,
  14.758463859558105,
  53.39189147949219,
  2.7885539531707764,
  14.758463859558105,
  14.758463859558105,
  14.758463859558105,
  44.663673400878906,
  4.89743709564209,
  1.6255521774291992,
  1.4945485591888428,
  14.758463859558105,
  1.5937821865081787,
  27.82819366455078,
  1.5922574996948242,
  36.944313049316406,
  13.64535903930664,
  2.7949717044830322,
  1.6255521774291992,
  36.944313049316406,
  14.758463859558105,
  14.758246421813965,
  84.42827606201172,
  1.5922574996948242,
  14.758463859558105,
  3.980473518371582,
  14.758463859558105,
  13.64535903930664,
  10.707064628601074,
  2.7885539531707764,
  4.280576229095459,
  6.078582763671875,
  39.093868255615234,
  2.813159227371216,
  14.758463859558105,
  24.007583618164062,
  14.7584638595581

## Reinforcement Learning

In [18]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    #tree_format = TreeFormat_all(new_dicto,symbol)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
    reward -= pred['pred'][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    #new_tree = np.array([str(TreeFormat_all(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
        reward -= pred['pred'][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        if q_val[act[0]] == 0:
            q_val[act[0]] = reward
        else:
            q_val[act[0]] = max(reward,q_val[act[0]])
    return q_value_neo

def RL_max_available_actions(q_value, old_states, actions):
    available_actions = RL_available_actions(actions, old_states)
    index_av = [i[0] for i in available_actions]
    q_value_acotado = q_value[index_av[0]:index_av[-1]+1]
    return np.max(q_value_acotado)

In [19]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

----------RL DATA----------
shape ds_rl : (2000, 67)
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


## MULTI-VAL

##### Seleccionar data

In [20]:
min_num_actions = -1
buenos_idx = []
aaa = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        aaa.append(y_rl_ind)
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_idx",total_buenos_idx)
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree',
            #'history_old',
            #'history_new'
             ]
 
iteration = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree'
             ]

total_idx 2000


##### Parameters

In [21]:
#epsilon = 0.7 ## EPSILON
#lr_rl = 0.9 ## LEARNING RATE
#disc_fac = 0.999 ## DISCOUNT FACTOR
#epoch_rl = 200 ### TOTAL EPOCH POR DATA
#n = model_name ##SALT Nombre archivos

### Q - Learning - multival

##### Parameters

In [22]:
epsilon_ql = epsilon ## EPSILON
lr_ql = lr_rl ## LEARNING RATE
disc_fac_ql = disc_fac ## DISCOUNT FACTOR
epoch_ql = epoch_rl ### TOTAL EPOCH POR DATA

##### Loop

In [23]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)
            
            new_state = len(current_state)-1
            old_q_value = q_value_ql[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_ql[new_state,:], old_states_val, actions)
            temporal_difference = reward + disc_fac_ql * q_max - old_q_value
            #temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

Start: 2022-01-04 04:54:10.089322
2022-01-04 04:54:59.315703 : 10/2000, index 9 evaluating
2022-01-04 04:55:59.364319 : 20/2000, index 19 evaluating
2022-01-04 04:56:58.425614 : 30/2000, index 29 evaluating
2022-01-04 04:57:55.153395 : 40/2000, index 39 evaluating
2022-01-04 04:58:53.479520 : 50/2000, index 49 evaluating
2022-01-04 04:59:56.142223 : 60/2000, index 59 evaluating
2022-01-04 05:00:43.607093 : 70/2000, index 69 evaluating
2022-01-04 05:01:47.964866 : 80/2000, index 79 evaluating
2022-01-04 05:02:39.976398 : 90/2000, index 89 evaluating
2022-01-04 05:03:36.699830 : 100/2000, index 99 evaluating
2022-01-04 05:04:44.872128 : 110/2000, index 109 evaluating
2022-01-04 05:05:51.831025 : 120/2000, index 119 evaluating
2022-01-04 05:06:50.744427 : 130/2000, index 129 evaluating
2022-01-04 05:07:44.520260 : 140/2000, index 139 evaluating
2022-01-04 05:08:43.022643 : 150/2000, index 149 evaluating
2022-01-04 05:09:36.898443 : 160/2000, index 159 evaluating
2022-01-04 05:10:41.677780

2022-01-04 07:09:10.024250 : 1370/2000, index 1369 evaluating
2022-01-04 07:10:12.245015 : 1380/2000, index 1379 evaluating
2022-01-04 07:11:12.909754 : 1390/2000, index 1389 evaluating
2022-01-04 07:12:26.599086 : 1400/2000, index 1399 evaluating
2022-01-04 07:13:33.456313 : 1410/2000, index 1409 evaluating
2022-01-04 07:14:28.063146 : 1420/2000, index 1419 evaluating
2022-01-04 07:15:27.227892 : 1430/2000, index 1429 evaluating
2022-01-04 07:16:29.839387 : 1440/2000, index 1439 evaluating
2022-01-04 07:17:28.939588 : 1450/2000, index 1449 evaluating
2022-01-04 07:18:56.331867 : 1460/2000, index 1459 evaluating
2022-01-04 07:19:59.238874 : 1470/2000, index 1469 evaluating
2022-01-04 07:21:04.941548 : 1480/2000, index 1479 evaluating
2022-01-04 07:22:07.068309 : 1490/2000, index 1489 evaluating
2022-01-04 07:23:07.576792 : 1500/2000, index 1499 evaluating
2022-01-04 07:24:06.614282 : 1510/2000, index 1509 evaluating
2022-01-04 07:25:05.836835 : 1520/2000, index 1519 evaluating
2022-01-

In [24]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_iteration = ql_df[iteration]
ql_df_iteration.to_csv(URL + 'rl_csvs/ql_df_iteration'+str(n)+'.csv',index=False)

In [25]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final'+str(n)+'.csv',index=False)

In [26]:
ql_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.244267,81.788513,49.0,49.0,52.479397,1075.086597,7.244267,32.788513,7.244267,32.788513,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.399632,82.269547,49.0,49.0,54.754546,1106.862725,7.399632,33.269547,7.399632,33.269547,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.241730,81.936249,49.0,49.0,52.442650,1084.796484,7.241730,32.936249,7.241730,32.936249,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.161949,56.405735,49.0,49.0,51.293516,54.844911,7.161949,7.405735,7.161949,7.405735,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.244678,56.562798,49.0,49.0,52.485367,57.195907,7.244678,7.562798,7.244678,7.562798,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.609123,30.749460,124.0,124.0,153.533827,8695.663169,12.390877,93.250540,12.390877,93.250540,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,110.941254,30.680840,124.0,124.0,170.530856,8708.465709,13.058746,93.319160,13.058746,93.319160,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,113.251854,30.865534,124.0,124.0,115.522644,8674.028789,10.748146,93.134466,10.748146,93.134466,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,109.801498,31.088608,124.0,124.0,201.597447,8632.526803,14.198502,92.911392,14.198502,92.911392,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [27]:
ql_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.175224,56.237526,49.0,49.0,51.483844,52.381782,7.175224,7.237526,7.175224,7.237526,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.760899,59.599762,46.0,46.0,162.840533,184.953525,12.760899,13.599762,12.760899,13.599762,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.410885,60.342957,48.0,48.0,154.030063,152.348576,12.410885,12.342957,12.410885,12.342957,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.918915,45.571594,48.0,48.0,4.330916,5.897155,2.081085,2.428406,2.081085,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.018356,102.969719,49.0,49.0,3846.276521,2912.730562,62.018356,53.969719,62.018356,53.969719,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,138.998047,138.608536,111.0,111.0,783.890629,762.231247,27.998047,27.608536,27.998047,27.608536,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,103.785019,59.439484,122.0,122.0,331.785536,3913.818207,18.214981,62.560516,18.214981,62.560516,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,131.974670,66.251312,109.0,109.0,527.835480,1827.450304,22.974670,42.748688,22.974670,42.748688,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.484100,71.748428,136.0,136.0,505.534768,4128.264460,22.484100,64.251572,22.484100,64.251572,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## RL-Neo Multival

###### Parameters

In [28]:
epsilon_neo = epsilon ## EPSILON
epoch_neo = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [29]:
values_neo = []
values_final_neo = []
c_neo = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_neo = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_neo % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_neo}/{total_buenos_idx}, index {idx} evaluating')
    ############# ALGORITMO NEO #####################3
    for i in range(0,epoch_neo):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_neo,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_neo)    
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_neo,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_neo)
        q_value_neo = RLNeo_q_values(q_value_neo,current_state, reward)
        ##############################################################
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_neo,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]
        values_neo.append(row)
    values_final_neo.append(row)
    c_neo += 1
#print("values_neo ", values_neo)
print("len(values_neo[0])",len(values_neo[0]) )
print("len(values_neo)", len(values_neo))

Start: 2022-01-04 08:17:42.788811
2022-01-04 08:18:20.664895 : 10/2000, index 9 evaluating
2022-01-04 08:19:01.204527 : 20/2000, index 19 evaluating
2022-01-04 08:19:43.198430 : 30/2000, index 29 evaluating
2022-01-04 08:20:25.211291 : 40/2000, index 39 evaluating
2022-01-04 08:21:07.250831 : 50/2000, index 49 evaluating
2022-01-04 08:21:49.603587 : 60/2000, index 59 evaluating
2022-01-04 08:22:31.045809 : 70/2000, index 69 evaluating
2022-01-04 08:23:13.552066 : 80/2000, index 79 evaluating
2022-01-04 08:23:55.252589 : 90/2000, index 89 evaluating
2022-01-04 08:24:37.190903 : 100/2000, index 99 evaluating
2022-01-04 08:25:19.773235 : 110/2000, index 109 evaluating
2022-01-04 08:26:02.398597 : 120/2000, index 119 evaluating
2022-01-04 08:26:44.529654 : 130/2000, index 129 evaluating
2022-01-04 08:27:26.280173 : 140/2000, index 139 evaluating
2022-01-04 08:28:08.214240 : 150/2000, index 149 evaluating
2022-01-04 08:28:49.918615 : 160/2000, index 159 evaluating
2022-01-04 08:29:32.458408

2022-01-04 09:54:05.807853 : 1370/2000, index 1369 evaluating
2022-01-04 09:54:48.302753 : 1380/2000, index 1379 evaluating
2022-01-04 09:55:30.767611 : 1390/2000, index 1389 evaluating
2022-01-04 09:56:14.198193 : 1400/2000, index 1399 evaluating
2022-01-04 09:56:56.990969 : 1410/2000, index 1409 evaluating
2022-01-04 09:57:38.975725 : 1420/2000, index 1419 evaluating
2022-01-04 09:58:21.340264 : 1430/2000, index 1429 evaluating
2022-01-04 09:59:03.901074 : 1440/2000, index 1439 evaluating
2022-01-04 09:59:46.236867 : 1450/2000, index 1449 evaluating
2022-01-04 10:00:30.804433 : 1460/2000, index 1459 evaluating
2022-01-04 10:01:13.412366 : 1470/2000, index 1469 evaluating
2022-01-04 10:01:56.281349 : 1480/2000, index 1479 evaluating
2022-01-04 10:02:38.785995 : 1490/2000, index 1489 evaluating
2022-01-04 10:03:21.328259 : 1500/2000, index 1499 evaluating
2022-01-04 10:04:03.635996 : 1510/2000, index 1509 evaluating
2022-01-04 10:04:45.857725 : 1520/2000, index 1519 evaluating
2022-01-

In [30]:
neo_df = pd.DataFrame(values_neo,columns=rl_columns)
neo_df_iteration = neo_df[iteration]
neo_df_iteration.to_csv(URL + 'rl_csvs/neo_df_iteration'+str(n)+'.csv',index=False)

In [31]:
neo_df_final = pd.DataFrame(values_final_neo,columns=rl_columns)
neo_df_final.to_csv(URL + 'rl_csvs/neo_df_final'+str(n)+'.csv',index=False)

In [32]:
neo_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.177242,56.197285,49.0,49.0,51.512807,51.800907,7.177242,7.197285,7.177242,7.197285,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,0,1,56.243122,56.226475,49.0,49.0,52.462818,52.221937,7.243122,7.226475,7.243122,7.226475,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.169769,56.139412,49.0,49.0,51.405592,50.971203,7.169769,7.139412,7.169769,7.139412,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.599285,56.192921,49.0,49.0,57.749134,51.738108,7.599285,7.192921,7.599285,7.192921,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.238808,56.593712,49.0,49.0,52.400337,57.664460,7.238808,7.593712,7.238808,7.593712,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,107.533401,30.919931,124.0,124.0,271.148867,8663.899168,16.466599,93.080069,16.466599,93.080069,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,110.551949,30.986919,124.0,124.0,180.850088,8651.433162,13.448051,93.013081,13.448051,93.013081,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,111.456459,30.809582,124.0,124.0,157.340420,8684.454052,12.543541,93.190418,12.543541,93.190418,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,110.735191,30.917313,124.0,124.0,175.955149,8664.386689,13.264809,93.082687,13.264809,93.082687,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [33]:
neo_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.374317,56.205219,49.0,49.0,54.380554,51.915185,7.374317,7.205219,7.374317,7.205219,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.592865,58.507538,46.0,46.0,158.580249,156.438503,12.592865,12.507538,12.592865,12.507538,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.477951,59.871677,48.0,48.0,155.699262,140.936724,12.477951,11.871677,12.477951,11.871677,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.274017,94.060768,49.0,49.0,3878.053235,2030.472824,62.274017,45.060768,62.274017,45.060768,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P885;ᶲIRI_ID;...dire..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.722687,80.943245,111.0,111.0,824.992735,903.408525,28.722687,30.056755,28.722687,30.056755,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1996,1996,199,100.743713,58.891125,122.0,122.0,451.829721,3982.730138,21.256287,63.108875,21.256287,63.108875,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.182129,74.289238,109.0,109.0,584.775358,1204.837000,24.182129,34.710762,24.182129,34.710762,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.720474,70.807861,136.0,136.0,516.219950,4250.014945,22.720474,65.192139,22.720474,65.192139,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Sarsa Multival

###### Parameters

In [34]:
epsilon_sar = epsilon ## EPSILON
lr_sar = lr_rl ## LEARNING RATE
disc_fac_sar = disc_fac ## DISCOUNT FACTOR
epoch_sar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [35]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            new_state = len(current_state)-1
            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

Start: 2022-01-04 10:39:21.602906
2022-01-04 10:40:15.030063 : 10/2000, index 9 evaluating
2022-01-04 10:41:14.710708 : 20/2000, index 19 evaluating
2022-01-04 10:42:14.988973 : 30/2000, index 29 evaluating
2022-01-04 10:43:13.013146 : 40/2000, index 39 evaluating
2022-01-04 10:44:12.756165 : 50/2000, index 49 evaluating
2022-01-04 10:45:16.827625 : 60/2000, index 59 evaluating
2022-01-04 10:46:05.667661 : 70/2000, index 69 evaluating
2022-01-04 10:47:11.294590 : 80/2000, index 79 evaluating
2022-01-04 10:48:04.662461 : 90/2000, index 89 evaluating
2022-01-04 10:49:02.553460 : 100/2000, index 99 evaluating
2022-01-04 10:50:11.899603 : 110/2000, index 109 evaluating
2022-01-04 10:51:20.432678 : 120/2000, index 119 evaluating
2022-01-04 10:52:20.620242 : 130/2000, index 129 evaluating
2022-01-04 10:53:15.549629 : 140/2000, index 139 evaluating
2022-01-04 10:54:14.834521 : 150/2000, index 149 evaluating
2022-01-04 10:55:09.566017 : 160/2000, index 159 evaluating
2022-01-04 10:56:15.719131

2022-01-04 12:55:18.512818 : 1370/2000, index 1369 evaluating
2022-01-04 12:56:21.310254 : 1380/2000, index 1379 evaluating
2022-01-04 12:57:22.313517 : 1390/2000, index 1389 evaluating
2022-01-04 12:58:36.218151 : 1400/2000, index 1399 evaluating
2022-01-04 12:59:43.870875 : 1410/2000, index 1409 evaluating
2022-01-04 13:00:39.084523 : 1420/2000, index 1419 evaluating
2022-01-04 13:01:38.882861 : 1430/2000, index 1429 evaluating
2022-01-04 13:02:41.804553 : 1440/2000, index 1439 evaluating
2022-01-04 13:03:41.749801 : 1450/2000, index 1449 evaluating
2022-01-04 13:05:09.539574 : 1460/2000, index 1459 evaluating
2022-01-04 13:06:12.464285 : 1470/2000, index 1469 evaluating
2022-01-04 13:07:18.960125 : 1480/2000, index 1479 evaluating
2022-01-04 13:08:21.828761 : 1490/2000, index 1489 evaluating
2022-01-04 13:09:23.367497 : 1500/2000, index 1499 evaluating
2022-01-04 13:10:22.992454 : 1510/2000, index 1509 evaluating
2022-01-04 13:11:23.024630 : 1520/2000, index 1519 evaluating
2022-01-

In [36]:
sarsa_df = pd.DataFrame(values_sar,columns=rl_columns)
sarsa_df_iteration = sarsa_df[iteration]
sarsa_df_iteration.to_csv(URL + 'rl_csvs/sarsa_df_iteration'+str(n)+'.csv',index=False)

In [37]:
sarsa_df_final = pd.DataFrame(values_final_sar,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final'+str(n)+'.csv',index=False)

In [38]:
sarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.238808,56.425774,49.0,49.0,52.400337,55.142114,7.238808,7.425774,7.238808,7.425774,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,0,1,56.330589,56.219624,49.0,49.0,53.737539,52.122964,7.330589,7.219624,7.330589,7.219624,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.265289,56.149818,49.0,49.0,52.784429,51.119903,7.265289,7.149818,7.265289,7.149818,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.226665,56.280746,49.0,49.0,52.224694,53.009269,7.226665,7.280746,7.226665,7.280746,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.249290,56.272499,49.0,49.0,52.552212,52.889243,7.249290,7.272499,7.249290,7.272499,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.548035,30.875586,124.0,124.0,155.051441,8672.156566,12.451965,93.124414,12.451965,93.124414,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.434479,31.010365,124.0,124.0,157.892324,8647.072304,12.565521,92.989635,12.565521,92.989635,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.897545,30.844921,124.0,124.0,171.674331,8677.868723,13.102455,93.155079,13.102455,93.155079,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,110.769562,30.759226,124.0,124.0,175.044496,8693.841965,13.230438,93.240774,13.230438,93.240774,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [39]:
sarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.177376,56.173943,49.0,49.0,51.514723,51.465452,7.177376,7.173943,7.177376,7.173943,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.698128,57.557613,46.0,46.0,161.242448,133.578427,12.698128,11.557613,12.698128,11.557613,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.188095,61.218800,48.0,48.0,148.549662,174.736663,12.188095,13.218800,12.188095,13.218800,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.905346,45.571594,48.0,48.0,4.387576,5.897155,2.094654,2.428406,2.094654,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.098236,114.079491,49.0,49.0,3856.190925,4235.340105,62.098236,65.079491,62.098236,65.079491,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.256500,138.786057,111.0,111.0,798.429806,772.064937,28.256500,27.786057,28.256500,27.786057,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,103.164391,59.184738,122.0,122.0,354.780183,3945.757120,18.835609,62.815262,18.835609,62.815262,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,132.491486,75.696686,109.0,109.0,551.849895,1109.110737,23.491486,33.303314,23.491486,33.303314,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.798187,72.654312,136.0,136.0,519.757342,4012.676171,22.798187,63.345688,22.798187,63.345688,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Expected-Sarsa Multival

###### Parameters

In [40]:
epsilon_exsar = epsilon ## EPSILON
epsilon_exsar_alg = epsilon
lr_exsar = lr_rl ## LEARNING RATE
disc_fac_exsar = disc_fac ## DISCOUNT FACTOR
epoch_exsar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [41]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_exsar[new_state,:], old_states_val, actions)
            #q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar_alg/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar_alg)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

Start: 2022-01-04 14:04:27.026265
2022-01-04 14:05:21.383711 : 10/2000, index 9 evaluating
2022-01-04 14:06:22.856744 : 20/2000, index 19 evaluating
2022-01-04 14:07:22.686265 : 30/2000, index 29 evaluating
2022-01-04 14:08:21.062143 : 40/2000, index 39 evaluating
2022-01-04 14:09:20.944994 : 50/2000, index 49 evaluating
2022-01-04 14:10:25.746007 : 60/2000, index 59 evaluating
2022-01-04 14:11:15.044195 : 70/2000, index 69 evaluating
2022-01-04 14:12:21.354621 : 80/2000, index 79 evaluating
2022-01-04 14:13:14.806370 : 90/2000, index 89 evaluating
2022-01-04 14:14:13.122986 : 100/2000, index 99 evaluating
2022-01-04 14:15:23.329734 : 110/2000, index 109 evaluating
2022-01-04 14:16:32.495671 : 120/2000, index 119 evaluating
2022-01-04 14:17:33.753260 : 130/2000, index 129 evaluating
2022-01-04 14:18:29.146374 : 140/2000, index 139 evaluating
2022-01-04 14:19:29.245858 : 150/2000, index 149 evaluating
2022-01-04 14:20:24.713820 : 160/2000, index 159 evaluating
2022-01-04 14:21:31.274158

2022-01-04 16:21:21.562510 : 1370/2000, index 1369 evaluating
2022-01-04 16:22:25.024958 : 1380/2000, index 1379 evaluating
2022-01-04 16:23:26.765133 : 1390/2000, index 1389 evaluating
2022-01-04 16:24:41.215548 : 1400/2000, index 1399 evaluating
2022-01-04 16:25:49.358769 : 1410/2000, index 1409 evaluating
2022-01-04 16:26:44.731470 : 1420/2000, index 1419 evaluating
2022-01-04 16:27:44.856321 : 1430/2000, index 1429 evaluating
2022-01-04 16:28:48.760554 : 1440/2000, index 1439 evaluating
2022-01-04 16:29:49.031834 : 1450/2000, index 1449 evaluating
2022-01-04 16:31:17.683994 : 1460/2000, index 1459 evaluating
2022-01-04 16:32:21.069441 : 1470/2000, index 1469 evaluating
2022-01-04 16:33:27.567067 : 1480/2000, index 1479 evaluating
2022-01-04 16:34:31.189290 : 1490/2000, index 1489 evaluating
2022-01-04 16:35:33.055939 : 1500/2000, index 1499 evaluating
2022-01-04 16:36:33.057537 : 1510/2000, index 1509 evaluating
2022-01-04 16:37:33.048263 : 1520/2000, index 1519 evaluating
2022-01-

In [42]:
exsarsa_df = pd.DataFrame(values_exsar,columns=rl_columns)
exsarsa_df_iteration = exsarsa_df[iteration]
exsarsa_df_iteration.to_csv(URL + 'rl_csvs/exsarsa_df_iteration'+str(n)+'.csv',index=False)

In [43]:
exsarsa_df_final = pd.DataFrame(values_final_exsar,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final'+str(n)+'.csv',index=False)

In [44]:
exsarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.211796,56.180729,49.0,49.0,52.009999,51.562868,7.211796,7.180729,7.211796,7.180729,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,0,1,56.249317,56.469028,49.0,49.0,52.552599,55.786386,7.249317,7.469028,7.249317,7.469028,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.178249,74.679558,49.0,49.0,51.527264,659.439689,7.178249,25.679558,7.178249,25.679558,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.399410,56.165192,49.0,49.0,54.751272,51.339971,7.399410,7.165192,7.399410,7.165192,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.683559,56.226910,49.0,49.0,59.037085,52.228223,7.683559,7.226910,7.683559,7.226910,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.778229,30.930944,124.0,124.0,149.371692,8661.849102,12.221771,93.069056,12.221771,93.069056,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,110.461830,31.124477,124.0,124.0,183.282043,8625.862701,13.538170,92.875523,13.538170,92.875523,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,111.659767,31.233513,124.0,124.0,152.281347,8605.621133,12.340233,92.766487,12.340233,92.766487,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,110.828575,31.139263,124.0,124.0,173.486433,8623.116448,13.171425,92.860737,13.171425,92.860737,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [45]:
exsarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.205441,56.259228,49.0,49.0,51.918373,52.696388,7.205441,7.259228,7.205441,7.259228,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.024063,59.301250,46.0,46.0,144.578094,176.923264,12.024063,13.301250,12.024063,13.301250,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.799877,60.242680,48.0,48.0,163.836855,149.883204,12.799877,12.242680,12.799877,12.242680,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,113.478218,111.659447,49.0,49.0,4157.440607,3926.206263,64.478218,62.659447,64.478218,62.659447,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.706573,139.728058,111.0,111.0,824.067361,825.301308,28.706573,28.728058,28.706573,28.728058,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,105.120079,58.802063,122.0,122.0,284.931732,3993.979243,16.879921,63.197937,16.879921,63.197937,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.596802,75.112534,109.0,109.0,605.002657,1148.360381,24.596802,33.887466,24.596802,33.887466,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.798111,71.959511,136.0,136.0,519.753863,4101.184257,22.798111,64.040489,22.798111,64.040489,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


In [46]:
1+1

2

### Archivos enteros

In [47]:
import os
path = "/home/school/math/final_analysis.csv"

In [48]:
os.remove(URL + 'rl_csvs/ql_df.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/ccarmona/memoria/dataset/rl_csvs/ql_df.csv'

In [ ]:
os.remove(URL + 'rl_csvs/sarsa_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/neo_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/exsarsa_df.csv')

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/sarsa_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/exsarsa_df.csv',index=False)